<a href="https://colab.research.google.com/github/Victor149286573/Hong-Kong-Film-Box-Office-Performance/blob/master/bridgewater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this part, we focus on
# Bridgewater Associates, LP: CIK Number - 0001350694


**Should Connect to the local notebook**
1.   jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888  --NotebookApp.port_retries=0
2.   Open the docker
3.   Connect to (Top right list) http://localhost:8888/....



In [ ]:
!pip install psycopg2 requests

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

In [ ]:
headers = {
    "User-Agent": "victor149286573@gmail.com",
    "Accept-Encoding": "gzip, deflate",
    "Host": "www.sec.gov"
}

In [ ]:
import psycopg2

# Connect to your local PostgreSQL database
def connect_to_database():
    try:
        connection = psycopg2.connect(
          host="localhost",
          database="postgres",
          user="postgres",
          password="example",
        )
        print("Connected to the database.")
        return connection
    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL:", error)
        return None

# Call the function to connect to the database
connection = connect_to_database()

Connected to the database.


In [ ]:
import requests

#How to get the website page url?
#From website page url, get the infotable
#get from function: get_links_for_cik
def extract_infotable_xml_url(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url,headers=headers) #headers=headers

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the anchor tag with href containing 'infotable.xml'
            infotable_link = soup.find('a', href=lambda href: href and ("xslForm13F_X02/infotable.xml" in href or "xslForm13F_X01/infotable.xml" in href or "xslForm13F_X01/form13fInfoTable.xml" in href))
            print(infotable_link)
            if infotable_link:
                infotable_url = "https://www.sec.gov"+infotable_link['href']
                print(f"URL for infotable.xml: {infotable_url}")
                return infotable_url
            else:
                print("infotable.xml not found in the HTML source.")
                return None
        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

url="https://www.sec.gov/Archives/edgar/data/0001350694000114036117031111/0001140361-17-031111-index.html"
extract_infotable_xml_url(url)

url_xml = extract_infotable_xml_url(url)

<a href="/Archives/edgar/data/1350694/000114036117031111/xslForm13F_X01/form13fInfoTable.xml">form13fInfoTable.html</a>
URL for infotable.xml: https://www.sec.gov/Archives/edgar/data/1350694/000114036117031111/xslForm13F_X01/form13fInfoTable.xml
<a href="/Archives/edgar/data/1350694/000114036117031111/xslForm13F_X01/form13fInfoTable.xml">form13fInfoTable.html</a>
URL for infotable.xml: https://www.sec.gov/Archives/edgar/data/1350694/000114036117031111/xslForm13F_X01/form13fInfoTable.xml


In [ ]:
import requests
import xml.etree.ElementTree as ET

headers = {
    "User-Agent": "victor149286573@gmail.com",
    "Accept-Encoding": "gzip, deflate",
    "Host": "www.sec.gov"
}

def get_links_for_cik(cik):
    base_url = "https://www.sec.gov/Archives/edgar/data/"
    url = f"{base_url}{cik}/index.xml"

    response = requests.get(url, headers=headers)
    xml_content = response.content.decode("UTF-8")

    # Parse the XML data
    root = ET.fromstring(xml_content)

    # Find all the "href" elements
    href_elements = root.findall(".//href")

    # Extract the links
    links = [elem.text for elem in href_elements]

    result_links = []

    # Process each link
    for link in links:
        # Split the URL by the last slash
        parts = link.rsplit("/", 1)
        last_part = parts[-1]

        # Parse into three parts
        part1 = last_part[:10]
        part2 = last_part[10:12]
        part3 = last_part[12:]

        if int(part2) < 15:
            print("Breaking the loop as Year is less than 15.")
            break

        new_link = f"https://www.sec.gov/Archives/edgar/data/{cik}/{last_part}/{part1}-{part2}-{part3}-index.html"
        result_links.append(new_link)

    return result_links

# Call the function with your CIK number
cik = "0001350694"  # Replace with your CIK number
links = get_links_for_cik(cik)

# Print the resulting links
for link in links:
    print(link)


Breaking the loop as Year is less than 15.
https://www.sec.gov/Archives/edgar/data/0001350694/000117266123002051/0001172661-23-002051-index.html
https://www.sec.gov/Archives/edgar/data/0001350694/000117266123000737/0001172661-23-000737-index.html
https://www.sec.gov/Archives/edgar/data/0001350694/000117266122002357/0001172661-22-002357-index.html
https://www.sec.gov/Archives/edgar/data/0001350694/000117266122001788/0001172661-22-001788-index.html
https://www.sec.gov/Archives/edgar/data/0001350694/000117266122001289/0001172661-22-001289-index.html
https://www.sec.gov/Archives/edgar/data/0001350694/000117266122000562/0001172661-22-000562-index.html
https://www.sec.gov/Archives/edgar/data/0001350694/000117266121002162/0001172661-21-002162-index.html
https://www.sec.gov/Archives/edgar/data/0001350694/000117266121001668/0001172661-21-001668-index.html
https://www.sec.gov/Archives/edgar/data/0001350694/000117266121001153/0001172661-21-001153-index.html
https://www.sec.gov/Archives/edgar/data

In [ ]:
headers

{'User-Agent': 'victor149286573@gmail.com',
 'Accept-Encoding': 'gzip, deflate',
 'Host': 'www.sec.gov'}

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_table_data(url):
    try:
        # Send an HTTP GET request and get the content of the page
        response = requests.get(url,headers=headers)
        response.raise_for_status()  # Raise an exception for 4xx/5xx status codes
        html_content = response.text

        # Parse the HTML using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the table in the HTML using the 'summary' attribute
        table = soup.find('table', summary='Form 13F-NT Header Information')

        if not table:
            print("Table not found on the page.")
            return None

        # Initialize a list to store the extracted data from the table
        data = []

        # Loop through the rows of the table (skipping the header row)
        for row in table.find_all('tr')[1:]:
            # Extract the data from each cell in the row
            cells = row.find_all('td')
            row_data = [cell.text.strip() for cell in cells]

            # Handle the case when the number of cells is 13 or 12

            if len(cells) == 13:
                selected_data = [row_data[0], row_data[2], row_data[4], row_data[5]]
            elif len(cells) == 12:
                selected_data = [row_data[0], row_data[2], row_data[3], row_data[4]]
            else:
                print("Unexpected number of cells in the row.")
                continue

            data.append(selected_data)

        # Create a DataFrame using the data and name the columns
        columns = ["NAME OF ISSUER", "CUSIP", "VALUE", "SHARE"]
        df = pd.DataFrame(data, columns=columns)

        return df

    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve the page. Error: {e}")
        return None

if __name__ == "__main__":
    # URL to fetch the HTML source code
    url = 'https://www.sec.gov/Archives/edgar/data/1350694/000117266123002051/xslForm13F_X02/infotable.xml'

    # Extract data from the table
    table_df = extract_table_data(url)

    # Print the extracted data
    if table_df is not None:
        print(table_df)
    else:
        print("Failed to extract data from the table.")


Unexpected number of cells in the row.
                   NAME OF ISSUER      CUSIP                    VALUE  \
0                  NAME OF ISSUER      CUSIP  (to the nearest dollar)   
1                360 DIGITECH INC  88557W101               23,092,363   
2                           3M CO  88579Y101                8,775,949   
3                     ABBOTT LABS  002824100              218,590,165   
4                      ABBVIE INC  00287Y109               65,569,599   
..                            ...        ...                      ...   
706              ZILLOW GROUP INC  98954M200               26,214,887   
707    ZIMMER BIOMET HOLDINGS INC  98956P102               37,929,244   
708                    ZOETIS INC  98978V103                6,804,067   
709  ZOOM VIDEO COMMUNICATIONS IN  98980L101                4,233,321   
710        ZTO EXPRESS CAYMAN INC  98980A105               22,118,584   

         SHARE  
0      PRN AMT  
1    1,190,328  
2       83,493  
3    2,158,702  

In [ ]:
def create_filings_table(connection):
    try:
        cursor = connection.cursor()

        create_table_query = """
        CREATE TABLE IF NOT EXISTS filings (
            id SERIAL PRIMARY KEY,
            company_name TEXT,
            filing_date DATE,
            filing_data JSONB
        );
        """
        cursor.execute(create_table_query)
        connection.commit()
        print("Table created successfully")

    except (Exception, psycopg2.Error) as error:
        print("Error while creating the table:", error)

# Call the function to create the table
create_filings_table(connection)

Table created successfully


In [ ]:
#Main

import requests
import xml.etree.ElementTree as ET
import pandas as pd

# Example: Set your CIK number
cik = "0001350694"

# Step 1: Get a list of URLs for infotable.xml files
links = get_links_for_cik(cik)

# Step 2-4: Fetch data and create DataFrames for each infotable.xml file
data_frames = []

for url in links:
    infotable = extract_infotable_xml_url(url)
    print(infotable)

    response = requests.get(infotable,headers=headers)
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the table rows (tr) in the HTML
    table_rows = soup.find_all("tr")

    # Extract the data from the table rows and create a list of dictionaries
    data = []
    for row in table_rows:
       # Find all the table cells (td) in each row
        cells = row.find_all("td")
        if len(cells) == 13:  # Ensure we have 13 cells per row (the correct table)
           row_data = [cell.text.strip() for cell in cells]
           data.append(row_data)

# Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data, columns=["Name of Issuer", "Title of Class", "CUSIP", "FIGI", "VALUE", "SHRS OR PRN AMT",
                                     "PRN", "CALL", "DISCRETION", "MANAGER", "SOLE", "SHARED", "NONE"])
    print(df)



Breaking the loop as Year is less than 15.
<a href="/Archives/edgar/data/1350694/000117266123002051/xslForm13F_X02/infotable.xml">infotable.html</a>
URL for infotable.xml: https://www.sec.gov/Archives/edgar/data/1350694/000117266123002051/xslForm13F_X02/infotable.xml
https://www.sec.gov/Archives/edgar/data/1350694/000117266123002051/xslForm13F_X02/infotable.xml
                   Name of Issuer   Title of Class      CUSIP  FIGI  \
0                  NAME OF ISSUER   TITLE OF CLASS      CUSIP  FIGI   
1                360 DIGITECH INC     AMERICAN DEP  88557W101     Â   
2                           3M CO              COM  88579Y101     Â   
3                     ABBOTT LABS              COM  002824100     Â   
4                      ABBVIE INC              COM  00287Y109     Â   
..                            ...              ...        ...   ...   
706              ZILLOW GROUP INC     CL C CAP STK  98954M200     Â   
707    ZIMMER BIOMET HOLDINGS INC              COM  98956P102     Â 

KeyboardInterrupt: ignored

In [ ]:
#Main

import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [ ]:
def generate_quarterly_dates():
    start_year = 2023
    end_year = pd.Timestamp.now().year
    quarters = [(year, quarter) for year in range(start_year, end_year + 1) for quarter in range(1, 5)]
    quarter_dates = [f"Q{quarter}-{year}" for year, quarter in quarters if year < end_year or quarter <= (pd.Timestamp.now().quarter)]
    return quarter_dates

In [ ]:
# Function to write DataFrame to an Excel file
def write_dataframe_to_excel(df, excel_file):
    with pd.ExcelWriter(excel_file) as writer:
        df.to_excel(writer, index=False)

#cik = 0001350694
# Sample usage
if __name__ == "__main__":
    # Get CIK from user input
    cik = input("Enter the CIK: ")
    excel_file_path = 'D:/3 Projects/wmin project/output.xlsx'

    links = get_links_for_cik(cik)
    for url in links:
        infotable = extract_infotable_xml_url(url)

        df = extract_table_data(infotable)
        print(df)
        if df is not None:
            # Write DataFrame into the Excel file
            write_dataframe_to_excel(df, excel_file_path)

            print("Data has been written to the Excel file.")

IndentationError: ignored

In [ ]:
import pandas as pd

# Function to generate quarterly dates from Q1-2023 to Q1-2015
def generate_quarterly_dates():
    quarters = [(year, quarter) for year in range(2023, 2014, -1) for quarter in range(1, 5)]
    quarter_dates = [f"Q{quarter}-{year}" for year, quarter in quarters]
    return quarter_dates

# Function to write DataFrame to an Excel file
def write_dataframe_to_excel(df, sheet_name, excel_file):
    with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a') as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Sample usage
if __name__ == "__main__":
    # Get CIK from user input
    cik = "0001350694"
#   cik = input("Enter the CIK: ")
    excel_file_path = 'D:/3 Projects/wmin project/output.xlsx'

    links = get_links_for_cik(cik)
    quarterly_dates = generate_quarterly_dates()

    # Create a new Excel file
    pd.DataFrame().to_excel(excel_file_path, index=False)

    for url, quarter_date in zip(links, quarterly_dates):
        infotable = extract_infotable_xml_url(url)
        df = extract_table_data(infotable)
        print(1)
        if df is not None:
            # Write DataFrame into the Excel file
            write_dataframe_to_excel(df, quarter_date, excel_file_path)

            print("Data has been written to the Excel file.")


Breaking the loop as Year is less than 15.
<a href="/Archives/edgar/data/1350694/000117266123002051/xslForm13F_X02/infotable.xml">infotable.html</a>
URL for infotable.xml: https://www.sec.gov/Archives/edgar/data/1350694/000117266123002051/xslForm13F_X02/infotable.xml
Unexpected number of cells in the row.
1
Data has been written to the Excel file.
<a href="/Archives/edgar/data/1350694/000117266123000737/xslForm13F_X02/infotable.xml">infotable.html</a>
URL for infotable.xml: https://www.sec.gov/Archives/edgar/data/1350694/000117266123000737/xslForm13F_X02/infotable.xml
Unexpected number of cells in the row.
1
Data has been written to the Excel file.
<a href="/Archives/edgar/data/1350694/000117266122002357/xslForm13F_X01/infotable.xml">infotable.html</a>
URL for infotable.xml: https://www.sec.gov/Archives/edgar/data/1350694/000117266122002357/xslForm13F_X01/infotable.xml
Unexpected number of cells in the row.
1
Data has been written to the Excel file.
<a href="/Archives/edgar/data/135069

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Function to extract data from the HTML table and return a DataFrame
def extract_table_data(url, quarter):
    try:
        response = requests.get(url,headers=headers)
        response.raise_for_status()
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the table in the HTML
        table = soup.find('table', summary='Form 13F-NT Header Information')
        if not table:
            print("Table not found on the page.")
            return None

        # Initialize a list to store the extracted data from the table
        data = []

        # Loop through the rows of the table
        for row in table.find_all('tr'):
            cells = row.find_all('td')
            # Check if it's a header row (number of cells == 11 or 12)
            if len(cells) in [11, 12]:
                columns = [cell.text.strip() for cell in cells]
                if len(columns) == 11:
                    # If 11 columns, add an empty value for the missing column
                    columns.insert(2, '')
                data.append(columns)
            elif len(cells) == 13:
                # If 13 columns, select the appropriate columns (0, 2, 4, 5)
                columns = [cells[0].text.strip(), cells[2].text.strip(), cells[4].text.strip(), cells[5].text.strip()]
                data.append(columns)

        df = pd.DataFrame(data[1:], columns=data[0])  # Skip the header row

        # Add the 'Quarter' column with the extracted quarter information
        df['Quarter'] = quarter

        # If the number of columns is less than 12, fill in empty values for the missing columns
        if len(df.columns) < 12:
            missing_columns = 12 - len(df.columns)
            for _ in range(missing_columns):
                df[f'Missing_{_+1}'] = ''

        return df

    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve the page. Error: {e}")
        return None

# Rest of the code remains the same...


# Function to generate quarterly dates from Q1-2023 to Q1-2015
def generate_quarterly_dates():
    quarters = []
    current_year = 2023
    current_quarter = 1

    while current_year >= 2015:
        quarter_str = f'Q{current_quarter}-{current_year}'
        quarters.append(quarter_str)

        current_quarter += 1
        if current_quarter > 4:
            current_quarter = 1
            current_year -= 1

    return quarters

# Rest of the code remains the same...

# Sample usage
if __name__ == "__main__":
    # Get CIK from user input
    cik = input("Enter the CIK: ")
    excel_file_path = 'D:/3 Projects/wmin project/output.xlsx'

    links = get_links_for_cik(cik)
    quarters = generate_quarterly_dates()
    i = 0

    # Initialize a DataFrame to store the combined data
    combined_df = pd.DataFrame()

    for url in links:
        infotable = extract_infotable_xml_url(url)

        df = extract_table_data(infotable, quarters[i])

        if df is not None:
            combined_df = combined_df.append(df, ignore_index=True)
            i += 1
            print(f"Data from {quarters[i]} has been extracted.")

    # Write the combined DataFrame into the Excel file
    write_dataframe_to_excel(combined_df, excel_file_path)

    print("Data has been written to the Excel file.")


Enter the CIK: 0001350694
Breaking the loop as Year is less than 15.
<a href="/Archives/edgar/data/1350694/000117266123002051/xslForm13F_X02/infotable.xml">infotable.html</a>
URL for infotable.xml: https://www.sec.gov/Archives/edgar/data/1350694/000117266123002051/xslForm13F_X02/infotable.xml


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd

# Function to read data from the Excel file
def read_data_from_excel(excel_file_path):
    # Read each sheet from the Excel file and store the data in a dictionary
    data_by_quarter = {}
    with pd.ExcelFile(excel_file_path) as xls:
        for sheet_name in xls.sheet_names:
            # Assume sheet names are in the format Q1-2023, Q4-2022, etc.
            quarter = sheet_name
            df = pd.read_excel(xls, sheet_name=sheet_name)
            data_by_quarter[quarter] = df
    return data_by_quarter

In [ ]:
# Function to write the merged data to a new Excel file
def write_merged_data_to_excel(merged_df, excel_file_path):
    # Write the merged DataFrame into a new Excel file
    with pd.ExcelWriter(excel_file_path) as writer:
        merged_df.to_excel(writer, index=False)

In [ ]:
# Function to merge data from all quarters
def merge_data(data_by_quarter):
    # Create a new DataFrame to store the merged data
    merged_df = pd.DataFrame()

    # Find common columns among all DataFrames
    common_columns = set.union(*[set(df.columns) for df in data_by_quarter.values()])

    # Check if 'NAME OF ISSUER' exists in the common columns
    if 'NAME OF ISSUER' not in common_columns:
        raise ValueError("Column 'NAME OF ISSUER' not found in the common columns.")

    # Iterate through the data_by_quarter dictionary and merge the data
    for quarter, df in data_by_quarter.items():
      # Filter the DataFrame to include only common columns
      common_columns_in_df = common_columns.intersection(df.columns)

      df = df[['NAME OF ISSUER','CUSIP','VALUE']]

      # Merge the data into the new DataFrame using the company name (NAME OF ISSUER) as the key
      if merged_df.empty:
         merged_df = df
      else:
         # Merge the data into the new DataFrame using the company name (NAME OF ISSUER) as the key
         merged_df = pd.merge(merged_df, df[['CUSIP','VALUE']], how='outer', on='CUSIP')


    return merged_df

# Sample usage
if __name__ == "__main__":
    excel_file_path = 'D:/3 Projects/wmin project/bridgewater.xlsx'

    # Step 1: Read data from the Excel file
    data_by_quarter = read_data_from_excel(excel_file_path)
    #print(data_by_quarter)
    # Step 2: Merge data from all quarters
    merged_df = merge_data(data_by_quarter)

    # Step 3: Write the merged data to a new Excel file
    new_excel_file_path = 'D:/3 Projects/wmin project/merged_output.xlsx'
    write_merged_data_to_excel(merged_df, new_excel_file_path)

    print("Data has been merged and written to the new Excel file.")


C:\Users\22600\AppData\Local\Temp\ipykernel_2292\2809170885.py:25: FutureWarning: Passing 'suffixes' which cause duplicate columns {'VALUE_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_df = pd.merge(merged_df, df[['CUSIP','VALUE']], how='outer', on='CUSIP')


Data has been merged and written to the new Excel file.


In [ ]:
    data_by_quarter = read_data_from_excel("D:/3 Projects/wmin project/bridgewater.xlsx")
    #print(data_by_quarter)

# Create a new DataFrame to store the merged data
    merged_df = pd.DataFrame()

    # Find common columns among all DataFrames
    common_columns = set.union(*[set(df.columns) for df in data_by_quarter.values()])
    print(common_columns)
    # Check if 'NAME OF ISSUER' exists in the common columns
    if 'NAME OF ISSUER' not in common_columns:
        raise ValueError("Column 'NAME OF ISSUER' not found in the common columns.")

    # Iterate through the data_by_quarter dictionary and merge the data
    for quarter, df in data_by_quarter.items():
      # Filter the DataFrame to include only common columns
      common_columns_in_df = common_columns.intersection(df.columns)

      df = df[common_columns_in_df]
      # Merge the data into the new DataFrame using the company name (NAME OF ISSUER) as the key
      if merged_df.empty:
         merged_df = df
      else:
         # Merge the data into the new DataFrame using the company name (NAME OF ISSUER) as the key
         merged_df = pd.merge(merged_df, df["VALUE"], how='left', on='NAME OF ISSUER')



{'CUSIP', 'NAME OF ISSUER', 'VALUE', 'SHARE'}
Empty DataFrame
Columns: []
Index: []
         CUSIP                NAME OF ISSUER                    VALUE  \
0        CUSIP                NAME OF ISSUER  (to the nearest dollar)   
1    88557W101              360 DIGITECH INC               23,092,363   
2    88579Y101                         3M CO                8,775,949   
3    002824100                   ABBOTT LABS              218,590,165   
4    00287Y109                    ABBVIE INC               65,569,599   
..         ...                           ...                      ...   
706  98954M200              ZILLOW GROUP INC               26,214,887   
707  98956P102    ZIMMER BIOMET HOLDINGS INC               37,929,244   
708  98978V103                    ZOETIS INC                6,804,067   
709  98980L101  ZOOM VIDEO COMMUNICATIONS IN                4,233,321   
710  98980A105        ZTO EXPRESS CAYMAN INC               22,118,584   

         SHARE  
0      PRN AMT  
1    

KeyError: ignored

In [ ]:
df1 = pd.read_excel("D:/3 Projects/wmin project/bridgewater.xlsx",sheet_name="Q1-2023")
df2 = pd.read_excel("D:/3 Projects/wmin project/bridgewater.xlsx",sheet_name="Q2-2023")
merged_df = pd.merge(df1, df2[['NAME OF ISSUER','VALUE']], how='left', on='NAME OF ISSUER')
print(merged_df)

                   NAME OF ISSUER                    VALUE
0                  NAME OF ISSUER  (to the nearest dollar)
1                10X GENOMICS INC                  475,323
2            1LIFE HEALTHCARE INC                  331,008
3                360 DIGITECH INC               25,306,218
4                           3M CO                3,448,420
..                            ...                      ...
816      ZIONS BANCORPORATION N A                2,659,605
817                    ZOETIS INC                4,055,039
818  ZOOM VIDEO COMMUNICATIONS IN                  483,799
819     ZOOMINFO TECHNOLOGIES INC                2,805,680
820        ZTO EXPRESS CAYMAN INC               36,849,464

[821 rows x 2 columns]
                   NAME OF ISSUER      CUSIP                  VALUE_x  \
0                  NAME OF ISSUER      CUSIP  (to the nearest dollar)   
1                360 DIGITECH INC  88557W101               23,092,363   
2                           3M CO  88579Y101     

In [ ]:
current_directory = os.getcwd()
current_directory

'C:\\Users\\22600'

In [ ]:
def write_dataframe_to_excel(dataframes, file_path):
    if not dataframes:
        print("No data to write. Please provide valid DataFrames.")
        return

    # Create a Pandas Excel writer object
    writer = pd.ExcelWriter(file_path, engine='xlsxwriter')

    # Get the current date
    current_date = datetime.now()

    # Sort the DataFrames in descending order of quarters
    dataframes.sort(key=lambda df: df['Date'].iloc[0], reverse=True)

    # Loop through the DataFrames and write each to a separate sheet
    for i, df in enumerate(dataframes):
        if df is not None:
            # Generate the quarterly label based on the date from the DataFrame
            date_str = df['Date'].iloc[0].strftime("%b-%Y")
            quarter_label = f"Q{df['Date'].iloc[0].quarter}-{date_str}"

            # Name the sheet with the quarterly label (e.g., Q1-2023, Q4-2022, etc.)
            sheet_name = f"Sheet_{quarter_label}"

            # Save the DataFrame to the Excel sheet
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Save and close the Excel writer
    writer.save()
    writer.close()

    # Get and print the current working directory
    current_directory = os.getcwd()
    print("Current working directory:", current_directory)

    print(f"DataFrames have been saved to {file_path}.")


if __name__ == "__main__":
    # ... (same as before)

    # List of URLs for different tables (replace with your actual URLs)
    urls = [
        'https://www.sec.gov/Archives/edgar/data/1350694/000117266123002051/xslForm13F_X02/infotable.xml',
        # Add more URLs here for different tables
    ]

    # List to store DataFrames
    dataframes = []

    # Loop through the URLs and extract data from each table
    for i, url in enumerate(urls):
        table_df = extract_table_data(url)
        if table_df is not None:
            dataframes.append(table_df)

    # Specify the desired path for saving the Excel file
    excel_file_path = 'C:/Users/YourUsername/Documents/output.xlsx'

    # Write the list of DataFrames to the Excel file
    write_dataframe_to_excel(dataframes, excel_file_path)

Unexpected number of cells in the row.


FileNotFoundError: ignored

In [ ]:
import os
os.getcwd()

'C:\\Users\\22600'

In [ ]:
    infotable = "https://www.sec.gov/Archives/edgar/data/1350694/000117266122002357/xslForm13F_X01/infotable.xml"

    response = requests.get(infotable,headers=headers)
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the table rows (tr) in the HTML
    table_rows = soup.find_all("tr")

    # Extract the data from the table rows and create a list of dictionaries
    data = []
    for row in table_rows:
       # Find all the table cells (td) in each row
        cells = row.find_all("td")
        print(cells)
        if len(cells) == 13:  # Ensure we have 13 cells per row (the correct table)
           row_data = [cell.text.strip() for cell in cells]
           data.append(row_data)
    print(data)
# Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data, columns=["Name of Issuer", "Title of Class", "CUSIP", "FIGI", "VALUE", "SHRS OR PRN AMT",
                                     "PRN", "CALL", "DISCRETION", "MANAGER", "SOLE", "SHARED", "NONE"])




In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def extract_info_table_data(url):
    try:
        # Send an HTTP GET request and get the content of the page
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for 4xx/5xx status codes
        html_content = response.text

        # Parse the HTML using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the table in the HTML using the 'summary' attribute
        table = soup.find('table', summary='Form 13F-NT Header Information')

        if not table:
            print("Table not found on the page.")
            return None

        # Initialize a list to store the extracted data from the table
        data = []

        # Loop through the rows of the table (skipping the header row)
        for row in table.find_all('tr')[1:]:
            # Extract the data from each cell in the row
            cells = row.find_all('td')
            row_data = [cell.text.strip() for cell in cells]
            data.append(row_data)

        # Create a DataFrame from the extracted data
        df = pd.DataFrame(data, columns=["NAME OF ISSUER", "TITLE OF CLASS", "CUSIP", "(x$1000)", "PRN AMT", "PRN",
                                         "CALL", "DISCRETION", "MANAGER", "SOLE", "SHARED", "NONE"])

        return df

    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve the page. Error: {e}")
        return None


# Function to write DataFrames into an Excel file with different sheets
def write_dataframes_to_excel(dataframes, excel_file):
    with pd.ExcelWriter(excel_file) as writer:
        for cik, df in dataframes.items():
            df.to_excel(writer, sheet_name=cik, index=False)

# Sample usage
if __name__ == "__main__":
    # URLs with InfoTable data for different CIKs
    cik_links = {
        "1350694": "https://www.sec.gov/Archives/edgar/data/1350694/000117266123002051/0001172661-23-002051-index.html",
        # Add more CIKs and links here
    }

    dataframes = {}
    for cik, link in cik_links.items():
        df = extract_info_table_data(link)
        if df is not None:
            dataframes[cik] = df

    # File path to save the Excel file
    excel_file_path = "info_table_data.xlsx"

    # Write DataFrames into the Excel file with different sheets
    write_dataframes_to_excel(dataframes, excel_file_path)

    print("Data has been written to the Excel file.")


In [ ]:
import pandas as pd

# Assuming you have a list of DataFrames named 'list_of_dataframes'
# Replace 'output_folder' with the desired folder path
output_folder = './output_folder/'

# Loop through the list of DataFrames and create separate Excel files for each DataFrame
for index, df in enumerate(list_of_dataframes):
    # Generate the file name for each DataFrame, for example: 'dataframe_0.xlsx', 'dataframe_1.xlsx', etc.
    output_file = f'{output_folder}dataframe_{index}.xlsx'

    # Create an ExcelWriter object to write to the Excel file
    with pd.ExcelWriter(output_file) as writer:
        # Write the DataFrame to the sheet named 'Sheet1'
        df.to_excel(writer, sheet_name='Sheet1', index=False)  # If you want to exclude the index column


# 提取index，提取index对应每一季度的website link，提取每个link中对应的table

In [ ]:
#store the data locally into a csv

import pandas as pd

# Send a request to fetch the HTML content
response = requests.get(url_xml,headers=headers)

 # Find all the table rows (tr) in the HTML
table_rows = soup.find_all("tr")

    # Extract the data from the table rows and create a list of dictionaries
data = []
for row in table_rows:
    # Find all the table cells (td) in each row
    cells = row.find_all("td")
    if len(cells) == 13:  # Ensure we have 13 cells per row (the correct table)
        row_data = [cell.text.strip() for cell in cells]
        data.append(row_data)

# Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data, columns=["Name of Issuer", "Title of Class", "CUSIP", "FIGI", "VALUE", "SHRS OR PRN AMT",
                                     "PRN", "CALL", "DISCRETION", "MANAGER", "SOLE", "SHARED", "NONE"])

# Print the DataFrame
print(df)
# Store the DataFrame in a CSV file locally
output_location = "D:\\3 Projects\\wmin project\\CSV outcomes\\bridgewater.csv"
df.to_csv(output_location, index=False)

print(f"DataFrame saved to '{output_location}'")



                   Name of Issuer   Title of Class      CUSIP  FIGI  \
0                  NAME OF ISSUER   TITLE OF CLASS      CUSIP  FIGI   
1                360 DIGITECH INC     AMERICAN DEP  88557W101         
2                           3M CO              COM  88579Y101         
3                     ABBOTT LABS              COM  002824100         
4                      ABBVIE INC              COM  00287Y109         
..                            ...              ...        ...   ...   
706              ZILLOW GROUP INC     CL C CAP STK  98954M200         
707    ZIMMER BIOMET HOLDINGS INC              COM  98956P102         
708                    ZOETIS INC             CL A  98978V103         
709  ZOOM VIDEO COMMUNICATIONS IN             CL A  98980L101         
710        ZTO EXPRESS CAYMAN INC  SPONSORED ADS A  98980A105         

                       VALUE SHRS OR PRN AMT  PRN  CALL  DISCRETION  MANAGER  \
0    (to the nearest dollar)         PRN AMT  PRN  CALL  DISCRETION

PermissionError: ignored

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

xml_content = []
headers = {
    "User-Agent": "victor149286573@gmail.com",
    "Accept-Encoding": "gzip, deflate",
    "Host": "www.sec.gov"
}
# Function to download 13F filings and store in the database
def download_and_store_filings(connection):
    base_url = "https://www.sec.gov/Archives/edgar/data/"
    cik = "0001350694"  # Replace with your CIK number
    url = f"{base_url}{cik}/index.xml"

    response = requests.get(url,headers=headers)
    xml_content = response.content.decode("UTF-8")
    print(xml_content)
   # print(response.content.decode("UTF-8"))
    if response.status_code == 200:

# Call the function to download and store filings
download_and_store_filings(connection)


TypeError: ignored

In [ ]:
if connection:
    connection.close()
    print("Database connection closed.")
